Setup Instructions
==========

These instructions install `miniconda` and the requisite libraries for the tutorial in a `conda` environment, which is an indepenent python installation.

Jupyter has the similar concept of _kernels_, which are independent execution environments. They don't even have to be Python, kernels for other languages exist as well.

By loading a separate kernel for each project, we avoid the complication of different components/projects having weird interactions, ultimately helping reproducibility.

We first produce a new conda environment with the libraries we require, then we tell Jupyter about this new environment with the ipython executable

NOTE: This step may take a little while

In [1]:
%%bash
#!/bin/bash

KERNEL_NAME="machine-learning-hats-2021"

set -e

# Download miniconda3 install script
if [[ ! -f $HOME/miniconda3.sh ]]; then
    wget --progress=dot:giga https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O $HOME/miniconda3.sh
fi

# Install miniconda3
if [[ ! -d $HOME/miniconda3 ]]; then
    bash $HOME/miniconda3.sh -b -f -u -p $HOME/miniconda3
    source $HOME/miniconda3/etc/profile.d/conda.sh
else
    source $HOME/miniconda3/etc/profile.d/conda.sh
fi

# Create conda environment
if [[ ! -d $HOME/miniconda3/envs/machine-learning-hats-2021 ]]; then
    conda install mamba -n base -c conda-forge
    mamba env create -f environment.yml --name $KERNEL_NAME
fi
conda activate $KERNEL_NAME

# Make a wrapper script to load miniconda python
if [[ ! -f python_wrapper.sh ]]; then
cat << 'EOF' > python_wrapper.sh
#!/bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate machine-learning-hats-2021
exec python "$@"
EOF
chmod +x python_wrapper.sh
fi

# Create the kernel
if [[ ! -d $HOME/.local/share/jupyter/kernels/$KERNEL_NAME ]]; then
mkdir -p "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME"
cat << EOF > "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME/kernel.json"
{
 "display_name": "$KERNEL_NAME", 
 "language": "python", 
 "argv": [
  "$PWD/python_wrapper.sh", 
  "-m", 
  "ipykernel_launcher", 
  "-f", 
  "{connection_file}"
 ]
}
EOF
fi

# Report OK
echo "Loaded $KERNEL_NAME!"

Solving environment: ...working... done

hdf5-1.10.6          | 3.1 MB    | ########## | 100% 
jupyterlab_pygments- | 8 KB      | ########## | 100% 
libogg-1.3.4         | 206 KB    | ########## | 100% 
keras-2.3.1          | 591 KB    | ########## | 100% 
cached_property-1.5. | 11 KB     | ########## | 100% 
libev-4.33           | 104 KB    | ########## | 100% 
uproot-base-4.0.11   | 165 KB    | ########## | 100% 
xz-5.2.5             | 343 KB    | ########## | 100% 
ca-certificates-2021 | 136 KB    | ########## | 100% 
numpy-1.21.0         | 6.1 MB    | ########## | 100% 
backports-1.0        | 4 KB      | ########## | 100% 
nest-asyncio-1.5.1   | 9 KB      | ########## | 100% 
pyqt-impl-5.12.3     | 5.9 MB    | ########## | 100% 
libtiff-4.3.0        | 668 KB    | ########## | 100% 
tqdm-4.61.2          | 80 KB     | ########## | 100% 
c-ares-1.17.1        | 109 KB    | ########## | 100% 
lz4-c-1.9.3          | 179 KB    | ########## | 100% 
gst-plugins-base-1.1 | 2.6 MB    | ######



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda




Results
=======

If successful, you should see something similar to the following:

```
Loaded machine-learning-hats-2021!
```

The new kernel you just made will then show up in the various Jupyter after refreshing the page.